# Platogram: Quick Start

## Introduction
Using this Notebook you can transform a an audio from YouTube or any source (Google Drive, Fathom, etc.) into paragraphs that spark joy. The resulting text is intended to be a lossless readable representation of the original audio, and will contain references to original transcript of the audio. You will also get summary and title.

Additionally you can store the results in the library and prompt it to generate new content with references to the original audio and transcript, or query it to retrieve paragraphs and underlying transcript.

We used the text of Chapter 1: Atoms in Motion from Volume 1. of Feynman's Lectures on Physics and a corresponding transcript obtained from https://www.feynmanlectures.caltech.edu/ as an example and reached out to Cal Tech for clarifications about fair use of the material. We observed high-quality output on single-speaker as well as multi-speaker audio in domains of eduction, family law, and business.

## Request for Feedback
We are looking for your feedback on the quality of the output for your use case and your suggestions on how to improve Platogram. Please reach out to us at platogram@codeanyway.com or ping Artyom in [Telegram](https://t.me/astaff239).

## Setup
To use this notebook you need:
1. Access to [Anthropic Claude](https://www.anthropic.com/claude) and API key ([create key](https://console.anthropic.com/settings/keys)).
2. (Optional) If subtitles are not available, you will need access to [Assembly AI](https://www.assemblyai.com/dashboard/signup) and API ([create key](https://www.assemblyai.com/app/)).

Ping on [Telegram](https://t.me/astaff239) or Discord.

## Costs
* Indexing is about $0.75 per hour of audio using Anthropic Claude 3.5 Sonnet model.
* Transcription is about $0.37 per hour of audio using Assembly AI Best model.

## Usage
* Update `URL`, `ANTHROPIC_API_KEY` and optionally `ASSEMBLYAI_API_KEY` below.
* Run the notebook by clicking "Run All" button above.

In [ ]:
#@title Install Platogram
# %%capture
# !pip install -e .
!sudo apt-get update && sudo apt-get install -y ffmpeg

# API Keys

* Set `ANTHROPIC_API_KEY` to your Anthropic API key: https://console.anthropic.com/settings/keys
* (Optional) Set `ASSEMBLYAI_API_KEY` to your Assembly AI API key: https://www.assemblyai.com/app/


In [10]:
ANTHROPIC_API_KEY="sk-ant-..." 
ASSEMBLYAI_API_KEY=None

# URL

* You can also use Dropbox, Fathom, etc as sources of audio.
* Set the `URL` to `"file:///path/to/file.mp3"` for local files.
* **Remember**: you can mount your Google Drive from Colab from File Browser and then use something like `"file:///content/drive/MyDrive/memos/01.mp3"`.

In [11]:
URL = "https://www.youtube.com/watch?v=W3I3kAg2J7w"

In [12]:
#@title Import Platogram and Initialize LLM and ASR Models
import platogram as plato


# Indexing is about $0.75 per hour of audio using Anthropic Claude 3.5 Sonnet model
llm = plato.llm.get_model("anthropic/claude-3-5-sonnet", ANTHROPIC_API_KEY)

# Transcription is about $0.37 per hour of audio using Assembly AI Best model
asr = plato.asr.get_model("assembly-ai/best", ASSEMBLYAI_API_KEY) if ASSEMBLYAI_API_KEY else None

In [ ]:
#@title Extract Transcript and create Index at about 1 minute per 12 minutes of audio.
%%capture

transcript = plato.extract_transcript(URL, asr)
content = plato.index(transcript, llm)

In [14]:
#@title Render Paragraphs and Transcript into Markdown

from IPython.display import display, Markdown
import re

from platogram.utils import format_time


def render_reference(i: int, first: int) -> str:
    link = f" [[{i-first+1}]]({URL}#t={content.transcript[i].time_ms // 1000})"
    return link


def render_transcript(first: int, last: int) -> str:
    return "<br>".join([
        f"{i+1}. <a href='{URL}#t={t.time_ms // 1000}'>{format_time(t.time_ms)}</a>: {t.text}"
        for i, t in enumerate(content.transcript[first:last+1])
    ])


def render_paragraph(p: str) -> str:
    references = sorted([int(i) for i in re.findall(r"【(\d+)】", p)])
    if not references:
        return p
    
    first = references[0] 
    last = references[-1]

    paragraph = re.sub(
        r"【(\d+)】",
        lambda match: render_reference(int(match.group(1)), first),
        p
    )
    transcript = render_transcript(first, last)
    return f"{paragraph}\n<details><summary>Transcript</summary>{transcript}</details>"

In [15]:
#@title Display Final Output

markdown = "\n\n".join([
    render_paragraph(p)
    for p in content.paragraphs
])

markdown = f"""
# {content.title}
{content.short_summary}
## Summary
{content.summary}
## Content
{markdown}
"""
display(Markdown(markdown))


# Navigating Dreams and Opportunities: A Nuanced Guide for Graduates
This commencement speech challenges the "follow your dreams" cliché, offering practical advice for career success. It emphasizes adding value to others, adapting to change, and viewing passion as a starting point rather than a fixed goal. The speaker encourages graduates to remain flexible, seek emerging opportunities, and recognize their potential to influence younger generations.
## Summary
In this commencement speech, the speaker, known for creating math visualization videos, offers a nuanced perspective on the common advice to "follow your dreams." He emphasizes the importance of adding value to others, recognizing that passion alone is not enough for career success. The speaker advises graduates to remain adaptable, seek opportunities in emerging fields, and understand that their interests and goals may change over time. He also highlights the impact one can have on younger generations and the importance of being responsive to a changing world. The speech concludes by encouraging graduates to view passion as a starting point rather than a fixed destination, and to focus on creating value, taking action, and remaining flexible in their pursuits.
## Content
Thank you, President Nemphard, for that very warm introduction and for inviting me. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=45) And thank you to the class of 2024 for including me in such a special day. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=49) I had the joy of getting to know many of you last year on this visit, and I distinctly remember coming away with the feeling that a future in your hands is a bright future indeed. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=53)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=45'>00:45</a>: Thank you, President Nemphard, for that very warm introduction and for inviting me.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=49'>00:49</a>: And thank you to the class of 2024 for including me in such a special day.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=53'>00:53</a>: I had the joy of getting to know many of you last year on this visit, and I distinctly remember coming away with the feeling that a future in your hands is a bright future indeed.</details>

For those in the audience who don't know who I am, I focus on making videos about mathematics with an emphasis on visualizations. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=62) It's a weird job. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=69) I do love it, though, and it's no exaggeration to describe it as a dream job. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=71) And a common cliche is for someone who was lucky enough to land in a dream job to stand confidently in front of a group of fledgling graduates and to compel them to follow their dreams. [[4]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=75) Frankly, on its own, I don't think this is very good advice. [[5]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=87)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=62'>01:02</a>: For those in the audience who don't know who I am, I focus on making videos about mathematics with an emphasis on visualizations.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=69'>01:09</a>: It's a weird job.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=71'>01:11</a>: I do love it, though, and it's no exaggeration to describe it as a dream job.<br>4. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=75'>01:15</a>: And a common cliche is for someone who was lucky enough to land in a dream job to stand confidently in front of a group of fledgling graduates and to compel them to follow their dreams.<br>5. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=87'>01:27</a>: Frankly, on its own, I don't think this is very good advice.</details>

To be clear, there is truth behind the cliche. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=93) It is true that those who make the biggest ripples are the ones who are fueled by passion. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=96) It is true that the life that you live is much more enjoyable if you can find something doing what you love. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=100) And it's also true that you shouldn't feel shackled by societal constraints. [[4]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=105) But for one thing, not everyone has a pre-baked dream sitting there waiting to be followed. [[5]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=110) That's completely okay. [[6]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=114) And even if you are one of the lucky ones who has a passion that you want to roll into a career, I think there are a few pragmatic concerns that don't always fit very neatly into an inspirational speech that are required to make this actually work. [[7]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=116)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=93'>01:33</a>: To be clear, there is truth behind the cliche.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=96'>01:36</a>: It is true that those who make the biggest ripples are the ones who are fueled by passion.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=100'>01:40</a>: It is true that the life that you live is much more enjoyable if you can find something doing what you love.<br>4. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=105'>01:45</a>: And it's also true that you shouldn't feel shackled by societal constraints.<br>5. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=110'>01:50</a>: But for one thing, not everyone has a pre baked dream sitting there waiting to be followed.<br>6. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=114'>01:54</a>: That's completely okay.<br>7. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=116'>01:56</a>: And even if you are one of the lucky ones who has a passion that you want to roll into a career, I think there are a few pragmatic concerns that don't always fit very neatly into an inspirational speech that are required to make this actually work.</details>

Now, I know I'm talking to a very nerdy audience, so I'm tempted to describe my aims here a little bit more mathematically. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=129) Precisely where, in the vector space of all possible advice, if you consider the "follow your dreams" vector, I want to explore its orthogonal subspace. Maybe, though, it's better if I just start with a story. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=134)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=129'>02:09</a>: Now, I know I'm talking to a very nerdy audience, so I'm tempted to describe my aims here a little bit more mathematically.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=134'>02:14</a>: Precisely where, in the vector space of all possible advice, if you consider the follow your dreams vector, I want to explore its orthogonal subspace maybe, though, it's better if I just start with a story.</details>

Before I entered college, I was one of those who knew what I wanted to major in. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=149) There's no surprises here. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=152) It was math. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=154) This was a topic that I had loved for a long time. [[4]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=154) As long as I can remember, when I was in college, I was plenty seduced by the adjacent field of computer science and programming, and I would spend my summers interning at software startups. [[5]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=156) But I distinctly remember coming back at the end of each of those summers and thinking, man, you know what I really want to do with my life is spend more time doing math. [[6]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=166)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=149'>02:29</a>: Before I entered college, I was one of those who knew what I wanted to major in.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=152'>02:32</a>: There's no surprises here.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=154'>02:34</a>: It was math.<br>4. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=154'>02:34</a>: This was a topic that I had loved for a long time.<br>5. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=156'>02:36</a>: As long as I can remember, when I was in college, I was plenty seduced by the adjacent field of computer science and programming, and I would spend my summers interning at software startups.<br>6. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=166'>02:46</a>: But I distinctly remember coming back at the end of each of those summers and thinking, man, you know what I really want to do with my life is spend more time doing math.</details>

So I had a passion. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=174) I had something I would want to follow. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=175) But in hindsight, that passion was a lot more arbitrary and maybe a little more self-centered than I would have liked to admit at the time. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=177) Why did I love math? [[4]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=184) If I'm honest, I think it had its roots in the fact that when I was younger, the adults emphasized, this is an important topic to learn, and they told me I was good at it. [[5]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=186) This makes me spend more time with it. [[6]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=194) Spending time with something is how you get better at something. [[7]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=195) And that kicks off a positive feedback loop in both senses of the word, positive feedback. [[8]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=198)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=174'>02:54</a>: So I had a passion.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=175'>02:55</a>: I had something I would want to follow.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=177'>02:57</a>: But in hindsight, that passion was a lot more arbitrary and maybe a little more self centered than I would have liked to admit at the time.<br>4. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=184'>03:04</a>: Why did I love math?<br>5. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=186'>03:06</a>: If I'm honest, I think it had its roots in the fact that when I was younger, the adults emphasized, this is an important topic to learn, and they told me I was good at it.<br>6. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=194'>03:14</a>: This makes me spend more time with it.<br>7. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=195'>03:15</a>: Spending time with something is how you get better at something.<br>8. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=198'>03:18</a>: And that kicks off a positive feedback loop in both senses of the word, positive feedback.</details>

Now, as time went on, I do believe it became less about perceptions. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=202) When I was in college, I remember genuinely enjoying the aesthetic delights that beautiful math problem solving has to have. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=206) But thinking of it as a career ambition, not just a hobby, this had the fatal flaw that I was viewing the world through a lens of what I personally enjoyed, not giving enough weight to a plan for how exactly it would add value to others. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=212)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=202'>03:22</a>: Now, as time went on, I do believe it became less about perceptions.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=206'>03:26</a>: When I was in college, I remember genuinely enjoying the aesthetic delights that beautiful math problem solving has to have.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=212'>03:32</a>: But thinking of it as a career ambition, not just a hobby, this had the fatal flaw that I was viewing the world through a lens of what I personally enjoyed, not giving enough weight to a plan for how exactly it would add value to others.</details>

I don't know if you felt it yet, but today marks a day in your life when a fundamental goal changes. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=226) When you're a student, the fundamental goal is to grow, to learn, to become better. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=231) So many institutions and structures around you are there to support you in growing and learning and getting better, and to reward you for doing so. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=236) In life after college, the goal changes a little, and success hinges on how effectively you're able to add value to others. [[4]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=243) Now, these aren't at odds with each other. [[5]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=251) In fact, they go hand in hand. [[6]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=253) You're much better positioned to make a difference if you're armed with an expertise and if you spend your life honing that expertise. [[7]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=255) But there is a big difference between personal growth being the end in and of itself, versus being a means to an end. [[8]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=260)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=226'>03:46</a>: I don't know if you felt it yet, but today marks a day in your life when a fundamental goal changes.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=231'>03:51</a>: When you're a student, the fundamental goal is to grow, to learn, to become better.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=236'>03:56</a>: So many institutions and structures around you are there to support you in growing and learning and getting better, and to reward you for doing so.<br>4. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=243'>04:03</a>: In life after college, the goal changes a little, and success hinges on how effectively you're able to add value to others.<br>5. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=251'>04:11</a>: Now, these aren't at odds with each other.<br>6. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=253'>04:13</a>: In fact, they go hand in hand.<br>7. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=255'>04:15</a>: You're much better positioned to make a difference if you're armed with an expertise and if you spend your life honing that expertise.<br>8. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=260'>04:20</a>: But there is a big difference between personal growth being the end in and of itself, versus being a means to an end.</details>

By way of comparison, I also loved the violin when I was growing up. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=267) And let's take a moment to imagine two distinct music students. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=271) I'm going to name them Paganini and Taylor. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=275) Both of them are talented, very talented. [[4]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=278) But Paganini pushes for technical excellence. [[5]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=280) He tries to perfect virtuosically challenging pieces. [[6]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=283) Taylor strives to write music that speaks to people, that resonates with them emotionally. [[7]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=286) Now, in a music school, [[8]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=292) Paganini is going to get the better grades every time. [[9]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=293) He's always going to get the better position. [[10]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=295) But pursuing music careers, Taylor's at the clear advantage. [[11]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=297)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=267'>04:27</a>: By way of comparison, I also loved the violin when I was growing up.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=271'>04:31</a>: And let's take a moment to imagine two distinct music students.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=275'>04:35</a>: I'm going to name them Paganini and Taylor.<br>4. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=278'>04:38</a>: Both of them are talented, very talented.<br>5. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=280'>04:40</a>: But Paganini pushes for technical excellence.<br>6. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=283'>04:43</a>: He tries to perfect virtuosically challenging pieces.<br>7. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=286'>04:46</a>: Taylor strives to write music that speaks to people, that resonates with them emotionally.<br>8. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=292'>04:52</a>: Now, in a music school.<br>9. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=293'>04:53</a>: Paganini is going to get the better grades every time.<br>10. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=295'>04:55</a>: He's always going to get the better position.<br>11. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=297'>04:57</a>: But pursuing music careers tailors at the clear advantage.</details>

My first piece of advice, something I would have told myself if I could go back in time and be in the seat where you are now, is that if you have a passion that you want to incorporate into a career, take a step back and recognize the fact that this is a passion that grew in a time of your life when the goal was to learn and to grow. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=302) But you're transitioning to a period when the primary aim shifts to adding value to others. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=318) The cliche to follow your dreams overlooks how critical it is that the dreams you have are about something more than just yourself. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=323)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=302'>05:02</a>: My first piece of advice, something I would have told myself I could go back in time and be in the seat where you are now, is that if you have a passion that you want to incorporate into a career, take a step back and recognize the fact that this is a passion that grew in a time of your life when the goal was to learn and to grow.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=318'>05:18</a>: But you're transitioning to a period when the primary aim shifts to adding value to others.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=323'>05:23</a>: The cliche to follow your dreams overlooks how critical it is that the dreams you have are about something more than just yourself.</details>

Those who excel in their first jobs are the ones who make life easier for everyone around them, even when it involves doing things they don't love. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=332) Those who excel in PhD programs are the ones who recognize how their work fits into a broader research community, not just the ones who view it as the next chapter in school. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=339) The successful entrepreneurs are the ones who have a relentless focus on making sure that what they have to sell is what people want to buy, not just those who are looking to make something impressive. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=348)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=332'>05:32</a>: Those who excel in their first jobs are the ones who make life easier for everyone around them, even when it involves doing things they don't love.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=339'>05:39</a>: Those who excel in PhD programs are the ones who recognize how their work fits into a broader research community, not just the ones who view it as the next chapter in school.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=348'>05:48</a>: The successful entrepreneurs are the ones who have a relentless focus on making sure that what they have to sell is what people want to buy, not just those who are looking to make something impressive.</details>

Now, for some people, when you hear the words "follow your dreams," it falls flat because you don't have a defining passion. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=357) Like I said earlier, that's completely okay. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=363) If anything, it might put you at an advantage. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=365) I think you'll do just as well if you start by seeking out opportunities where the skills that you've developed here intersect with adding values to others, and from there, I promise the passion will follow. [[4]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=368)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=357'>05:57</a>: Now, for some people, when you hear the words follow your dreams, it falls flat because you don't have a defining passion.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=363'>06:03</a>: Like I said earlier, that's completely okay.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=365'>06:05</a>: If anything, it might put you at an advantage.<br>4. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=368'>06:08</a>: I think you'll do just as well if you start by seeking out opportunities where the skills that you've developed here intersect with adding values to others, and from there, I promise the passion will follow.</details>

One of the best pieces of advice I remember receiving from a friend many years ago is that action precedes motivation. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=379) This is often useful on a much smaller scale. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=386) We feel most awake after getting out of bed, not before. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=389) A drive to exercise comes from the habit of exercising. [[4]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=392) It doesn't go the other way around. [[5]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=395) But I think the idea that action precedes motivation applies to this bigger question of finding a career doing what you love. [[6]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=396)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=379'>06:19</a>: One of the best pieces of advice I remember receiving from a friend many years ago is that action precedes motivation.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=386'>06:26</a>: This is often useful on a much smaller scale.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=389'>06:29</a>: We feel most awake after getting out of bed, not before.<br>4. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=392'>06:32</a>: A drive to exercise comes from the habit of exercising.<br>5. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=395'>06:35</a>: It doesn't go the other way around.<br>6. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=396'>06:36</a>: But I think the idea that action precedes motivation applies to this bigger question of finding a career doing what you love.</details>

These days, I do love making videos, and I really do love teaching. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=403) But when I was finishing college, I had no penchant or experience with videos at all, and my interest in teaching was honestly only insofar as it scratched this itch to do more math. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=406) It was only by stumbling into a wacky career where I was doing both of them that I came to love them. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=417)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=403'>06:43</a>: These days, I do love making videos, and I really do love teaching.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=406'>06:46</a>: But when I was finishing college, I had no penchant or experience with videos at all, and my interest in teaching was honestly only insofar as it scratched this itch to do more math.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=417'>06:57</a>: It was only by stumbling into a wacky career where I was doing both of them that I came to love them.</details>

There's a post I like on the webcomic XKCD that shows a man standing on a stage surrounded by bags of cash. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=435) He says, "Never stop buying lottery tickets. No matter what people tell you, I failed again and again, but I never gave up. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=442) [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=444) And here I am as proof that if you put in the time, it pays off." [[4]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=448) The caption notes that every inspirational speech should come with a disclaimer about survivorship bias. [[5]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=453)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=435'>07:15</a>: There's a post I like on the webcomic XKCD that shows a man standing on a stage and he has bags of cash surrounding him.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=442'>07:22</a>: Never stop buying lottery tickets, he says.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=444'>07:24</a>: No matter what people tell you, I failed again and again, but I never gave up.<br>4. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=448'>07:28</a>: And here I am as proof that if you put in the time, it pays off.<br>5. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=453'>07:33</a>: The caption notes that every inspirational speech should come with a disclaimer about survivorship bias.</details>

The obvious way that "follow your dreams" is susceptible to survivorship bias is that for all high-risk, high-reward paths - like professional athletics, starting a social media company, or making a career in the arts - only the few who rise to the top are in a position to give advice at all. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=460) But there's also a more subtle way that survivorship bias applies here. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=476) It's not just about the odds of winning a particular game; it has to do with whether the game you choose to play meshes well with the way that the future unfolds. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=480)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=460'>07:40</a>: The obvious way that follow your dreams is susceptible to survivorship bias is that for all of the high risk, high reward paths, things like professional athletics, starting a social media company, making a career in the arts, it's only the few who rise to the top who are in a position to give advice at all.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=476'>07:56</a>: But there's also a more subtle way that survivorship bias applies here.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=480'>08:00</a>: It's not just about the odds of winning a particular game, it has to do with whether the game you choose to play meshes well with the way that the future unfolds.</details>

If you were a software enthusiast in the late 1980s, you would be well-poised to ride the dot-com boom in the decade that followed. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=490) When I was finishing my undergrad, one of the ways I scratched that itch to do more math was to hack together a very rudimentary Python library for making math visualizations, which I used to make a couple of videos about neat proofs and problems that I enjoyed, posting them online. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=497) I was not planning for this to be a career. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=522) I had an appreciation for how valuable personal projects are, but it didn't go much beyond that. [[4]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=524)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=490'>08:10</a>: If you were a software enthusiast in the late 1980s, you would be well poised to ride the.com boom in the decade that followed.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=497'>08:17</a>: If you were someone with a niche interest in an act for film production, you would find yourself with an unexpected opportunity when YouTube and other film sharing platforms started to rise in prominence when I was finishing my undergrad, one of these ways that I scratched that itch to do more math was to hack together a very rudimentary python library for making math visualizations, and I used it to make a couple videos about neat proofs and problems that I enjoyed and posting them online.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=522'>08:42</a>: I was not planning for this to be a career.<br>4. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=524'>08:44</a>: I had an appreciation for how valuable personal projects are, but it didn't go much beyond that.</details>

This led to conversations with Khan Academy, a group I had great respect for, and it turned into a job there, making more lessons online. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=529) In the meantime, I continued my own channel as a side hobby, and it didn't blow up. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=536) But there was a very modest growth of others who enjoyed the same kind of visualizations that I did, and I saw just a steady take-up in the audience size. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=541)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=529'>08:49</a>: This led to conversations with Khan Academy, a group I had great respect for, and it turned into a job there, making more lessons online.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=536'>08:56</a>: In the meantime, I continued my own channel as a side hobby, and it didn't blow up.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=541'>09:01</a>: But there was a very modest growth of others who enjoyed the same kind of visualizations that I did, and I saw it in just a steady take up in the audience size.</details>

My original plan was to spend a year or two doing this online education stuff, working at Khan Academy and maybe returning to do a PhD. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=549) But as time went on, something between the gratitude that I saw from many students around the planet for the lessons I'd put out [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=557) and the slow and steady growth on my own channel led me to double down and form a somewhat unorthodox career in online lessons and math visualization. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=564)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=549'>09:09</a>: Now, my original plan, I think, was to spend a year or two doing this online education stuff, working at Khan Academy and maybe returning to do a PhD.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=557'>09:17</a>: But as time went on, something between the gratitude that I saw from many students around the planet for the lessons I'd put out.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=564'>09:24</a>: And the slow and steady growth on my own channel led me to doubling down and forming a somewhat unorthodox career in online lessons and math visualization.</details>

Looking back, it would feel very incomplete if I were to somehow ascribe the success that I found (to the extent there was any) to the fact that I was following a dream, pursuing a passion. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=572) Passion plays into it. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=582) You can't have good lessons without a teacher who cares. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=583) But we can't ignore the other factors at play. [[4]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=586) [[5]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=588) Success is a function of the value you bring to others. [[6]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=590) So a pursuit equally fueled by love, but which did nothing to help or to entertain people, just wouldn't have had a chance to work. [[7]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=592)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=572'>09:32</a>: Now, looking back, it would feel very incomplete if I were to somehow ascribe the success that I found to the extent there was any to the fact that I was following a dream, pursuing a passion.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=582'>09:42</a>: Passion plays into it.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=583'>09:43</a>: You can't have good lessons without a teacher who cares?<br>4. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=586'>09:46</a>: But we can't ignore the other factors at play.<br>5. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=588'>09:48</a>: I already brought up the biggest one.<br>6. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=590'>09:50</a>: Success is a function of the value you bring to others.<br>7. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=592'>09:52</a>: So a pursuit equally fueled by love, but which did nothing to help or to entertain people, just wouldn't have had a chance to work.</details>

Another factor I want to focus on is how I was very lucky with the timing. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=599) If I had been born ten years earlier, I don't think I could have reached the same number of people posting lessons on a much more infant version of the Internet, where there was less infrastructure that could have existed to help form a career doing so. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=604) If I had started ten years later, the space would have been a lot more saturated. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=616)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=599'>09:59</a>: But another factor I want to focus on is how I was very lucky with the timing.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=604'>10:04</a>: If I had been born ten years earlier, I don't think I could have reached the same number of people posting lessons on a much more infant version of the Internet, where there was less infrastructure that could have existed to help form a career doing so.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=616'>10:16</a>: If I had started ten years later, the space would have been a lot more saturated.</details>

So another piece of advice that I'd like to offer, another little ingredient that makes following your dreams a little more likely to work out, is to ask yourself what's possible now that wasn't possible ten years ago, and which might get harder ten years from now. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=620) There are more opportunities in a less crowded landscape. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=634) There are more chances to grow if you're part of a rising tide. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=636) But this requires pushing past the inevitable discomfort that comes from following a path that has little to no precedent. [[4]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=639)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=620'>10:20</a>: So another piece of advice that I'd like to offer, another little ingredient that makes following your dreams a little more likely to work out, is to ask yourself what's possible now that wasn't possible ten years ago, and which might get harder ten years from now.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=634'>10:34</a>: There are more opportunities in a less crowded landscape.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=636'>10:36</a>: There are more chances to grow if you're part of a rising tide.<br>4. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=639'>10:39</a>: But this requires pushing past the inevitable discomfort that comes from following a path that has little to no precedent.</details>

Next, I want to take a moment to talk about whose dreams you should be thinking about, because it's not just your own. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=647) When I was visiting Harvey Mudd last year, I had the pleasure of talking to one of the gems in your math department, Talithia Williams, and I asked her what made her pursue math in the first place. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=654) She had a very clear story. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=664) She told me she hadn't thought about it very much until one distinct day in her high school calculus class. [[4]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=665) Her teacher, Mr. Dorman, pulled her aside and said, "Talithia, you're really good at this. [[5]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=670) You should consider majoring in math." [[6]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=675) Evidently, she had never thought about it before, but that one comment was enough to knock over the first in a series of dominoes that led to a very flourishing career in the topic. [[7]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=677)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=647'>10:47</a>: Now, next, I want to take a moment to talk about whose dreams you should be thinking about, because it's not just your own.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=654'>10:54</a>: When I was visiting Harvey Mudd last year, I had the pleasure of talking to one of the gems in your math department, Talithia Williams, and I asked her, hey, what made you pursue math in the first place?<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=664'>11:04</a>: She had a very clear story.<br>4. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=665'>11:05</a>: She told me she hadn't thought about it very much until one distinct day in her high school calculus class.<br>5. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=670'>11:10</a>: Her teacher, Mister Dorman, pulled her aside and said, Talithia, you're really good at this.<br>6. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=675'>11:15</a>: You should consider majoring in math.<br>7. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=677'>11:17</a>: Evidently, she had never thought about it before, but that one comment was enough to knock over the first in a series of dominoes that led to a very flourishing career in the topic.</details>

Over the years, I've asked a lot of mathematicians the same question, and you would be shocked how often I hear a very similar answer. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=686) There was this one particular teacher and one seemingly simple thing that they did. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=691) That was the beginning of a long series of encouragements. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=696) Never underestimate just how much influence you can have on others, especially the ones who are younger than you are. [[4]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=700) Growing older is a process of slowly seeing the proportion of people around you who are younger than you rise inexorably closer to 100%. [[5]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=707) As this happens, you stand to have as much influence by shaping the dreams of those behind you as you do by following those of your own. [[6]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=717)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=686'>11:26</a>: Over the years, I've asked a lot of mathematicians the same question, and you would be shocked how often I hear a very similar answer.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=691'>11:31</a>: There was this one particular teacher and one seemingly simple thing that they did.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=696'>11:36</a>: That was the beginning in a long series of encouragements.<br>4. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=700'>11:40</a>: Never underestimate just how much influence you can have on others, especially the ones who are younger than you are.<br>5. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=707'>11:47</a>: Growing older is a process of slowly seeing the proportion of people around you who are younger than you are rise inexorably closer to 100%.<br>6. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=717'>11:57</a>: As this happens, you stand to have as much influence by shaping the dreams of those behind you as you do by following those of your own.</details>

As a very last point, the biggest risk in the "follow your dreams" cliché is the implication that there's one static target point at all. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=726) In the next 10, 20, 30 years, the world around you is going to change a lot, and those changes are going to be unpredictable. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=735) I hardly need to emphasize this point. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=744) You are the class who spent your formative transition from high school to college under a pandemic. [[4]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=746) But it's not just the world around you. [[5]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=752)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=726'>12:06</a>: And as a very last point, the biggest risk in the follow your dreams cliche is the implication that there's one static target point at all.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=735'>12:15</a>: In the next 10, 20, 30 years, the world around you is going to change a lot, and those changes are going to be unpredictable.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=744'>12:24</a>: I hardly need to emphasize this point.<br>4. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=746'>12:26</a>: You are the class who spent your formative transition from high school to college under a pandemic.<br>5. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=752'>12:32</a>: But it's not just the world around you.</details>

Tonight, when you're celebrating your graduation, and hopefully remembering to celebrate Mother's Day as well, take a few moments to ask the people who are older than you how they've changed, how their personalities and value systems have changed since they were students. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=754) You'll notice that essentially all of them have an answer which suggests you have every reason to expect that there's going to be something fundamental about you that changes as well in the coming decades. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=768) Probably unpredictably, almost everyone I know has undergone some kind of shift since college. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=777) Some came to place more value on having a family than they used to, some shifted from a trajectory that was oriented towards an academic career to going into industry. [[4]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=784) Some went the other way around and after spending some time in industry, returned to grad school. [[5]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=793) And so many of them have jobs that simply didn't exist at the time of their graduation. [[6]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=797)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=754'>12:34</a>: Tonight, when you're celebrating your graduation, and hopefully remembering to celebrate Mother's day as well, take a few moments to ask the people who are older than you how they've changed, how their personalities, how their value systems have changed since they were a student.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=768'>12:48</a>: You'll notice that essentially all of them has an answer which suggests you have every reason to expect that there's going to be something fundamental about you that changes as well in the coming decades.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=777'>12:57</a>: Probably unpredictably, almost everyone I know has undergone some kind of shift since college.<br>4. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=784'>13:04</a>: Some came to place more value on having a family than they used to, some shifted from a trajectory that was oriented towards an academic career to going into industry.<br>5. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=793'>13:13</a>: Some went the other way around and after spending some time in industry, returned to grad school.<br>6. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=797'>13:17</a>: And so many of them have jobs that simply didn't exist at the time of their graduation.</details>

So rather than having any one particular goal that defines who you are, you'll take better advantage of whatever the future has to offer you [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=803) if you remain nimble, if you're responsive to the changes in the world, and if you anticipate change within yourself. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=811) My final piece of advice is to not treat passion as something to follow. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=818) Think of it as an initial velocity vector. [[4]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=822) It gives a clear direction to point yourself, and loving what you do can have you move quickly. [[5]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=825) But you should expect, and you should even hope, that the specific direction that you're moving changes based on the force vectors around you. [[6]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=831)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=803'>13:23</a>: So rather than having any one particular goal that defines who you are, you'll take better advantage of whatever the future has to offer you.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=811'>13:31</a>: If you remain nimble, and if you're responsive to the changes in the world, and if you anticipate change within yourself.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=818'>13:38</a>: My final piece of advice is to not treat passion as something to follow.<br>4. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=822'>13:42</a>: Think of it as an initial velocity vector.<br>5. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=825'>13:45</a>: It gives a clear direction to point yourself, and loving what you do can have you move quickly.<br>6. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=831'>13:51</a>: But you should expect, and you should even hope, that the specific direction that you're moving changes based on the force vectors around you.</details>

If you step into the next chapter of life with an implacable focus on adding value to others, you're more likely to be at the helm. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=863) If you recognize that action precedes motivation, you're more likely to be at the helm. [[2]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=870) And if you ask what's possible now that wasn't ten years ago, you're more likely to be at the helm. [[3]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=875) If you appreciate just how much power you have to shape the lives of the generation that follows you, you are more likely to be at the helm. [[4]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=881) And if you remain adaptable to a changing world, treating passion not as a destination but as a fuel, following not dreams but opportunities, you're more likely to be at the helm. [[5]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=888)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=863'>14:23</a>: If you step into the next chapter of life with an implacable focus on adding values to others, you're more likely to be the ones at the helm.<br>2. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=870'>14:30</a>: If you recognize that action precedes motivation, you're more likely to be at the helm.<br>3. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=875'>14:35</a>: And if you ask what's possible now that wasn't ten years ago, you're more likely to be at the helm.<br>4. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=881'>14:41</a>: If you appreciate just how much power you have to shape the lives of the generation that follows you, you are more likely to be at the helm.<br>5. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=888'>14:48</a>: And if you remain adaptable to a changing world, treating passion not as a destination, but as a fuel, following not dreams, but opportunities, you're more likely to be at the helm.</details>

One final time, would everyone please join me in congratulating the class of 2024 on what they've done to get here and make some noise to let them know how excited we are to see where they go from here. [[1]](https://www.youtube.com/watch?v=W3I3kAg2J7w#t=899)
<details><summary>Transcript</summary>1. <a href='https://www.youtube.com/watch?v=W3I3kAg2J7w#t=899'>14:59</a>: One final time, would everyone please join me in congratulating the class of 2024 on what they've done to get here and make some noise to let them know how excited we are to see where they go from here.</details>


# Prompt your content

Use your content to create context and prompt it:
* Set `PROMPT` 
* Set `CONTEXT_SIZE`:
- `"small"` - only paragraphs will be used.
- `"medium"` - only transcript will be used.
- `"large"` - both paragraphs and transcript will be used.

In [ ]:
#@title Prompt your context


PROMPT = "Suggest an extended opening line to a conversation about this <context>."
CONTEXT_SIZE = "large"
response = llm.prompt(
    prompt=PROMPT,
    context=[content],
    context_size=CONTEXT_SIZE,
)
print(response)